## Import Packages

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from google.colab import drive
from tqdm import tqdm
from timm import create_model
from torch.utils.data import DataLoader

## Mount the Google Drive

In [ ]:
# Google Drive를 '/content/drive' 경로에 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## CNN and ViT Models

In [ ]:
# ViT-L-16 모델 로드
model_vit = create_model('vit_large_patch16_224', pretrained=True)

# 모델의 분류 헤드를 원하는 클래스 수로 수정 (예: CIFAR-10 -> 10 클래스)
model_vit.head = torch.nn.Linear(model_vit.head.in_features, 10)  # CIFAR-10의 10 클래스

# GPU 또는 CPU로 모델 전송
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_vit = model_vit.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
# ResNet-18 로드 (사전 학습된 가중치 사용)
model_cnn = models.resnet18(pretrained=True)

# CIFAR-10에 맞게 마지막 분류 계층 수정
model_cnn.fc = nn.Linear(model_cnn.fc.in_features, 10)

# 모델을 GPU로 이동
model_cnn = model_cnn.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 172MB/s]


## Load the dataset

In [ ]:
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT에 맞게 크기 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # 정규화
])

# CIFAR-10 데이터셋 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:01<00:00, 103MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# 데이터셋 로드 확인
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 50000
Number of test samples: 10000


## Model Train

In [ ]:
# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer_vit = optim.Adam(model_vit.parameters(), lr=0.001)
optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=0.001)

In [ ]:
# 학습 함수
def train_model(model, optimizer, train_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        print(f"Epoch {epoch+1}/{num_epochs}:")

        # tqdm으로 학습 상태 표시
        batch_progress = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch_idx, (images, labels) in batch_progress:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 손실 누적 및 미니 배치 손실 업데이트
            running_loss += loss.item()
            batch_progress.set_postfix(loss=loss.item())  # tqdm에 손실 표시

        # 에포크 종료 후 평균 손실 출력
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1} Completed, Average Loss: {epoch_loss:.4f}")
        print("-" * 50)

In [ ]:
model_vit = model_vit.to(device)
model_cnn = model_cnn.to(device)

In [ ]:
# ViT 학습
train_model(model_vit, optimizer_vit, train_loader, num_epochs=10)

# CNN 학습
train_model(model_cnn, optimizer_cnn, train_loader, num_epochs=10)

Epoch 1/10:


Epoch 1/10: 100%|██████████| 782/782 [10:30:15<00:00, 48.36s/it, loss=2]


Epoch 1 Completed, Average Loss: 1.9630
--------------------------------------------------
Epoch 2/10:


Epoch 2/10:   5%|▍         | 38/782 [32:46<10:41:46, 51.76s/it, loss=1.63]


KeyboardInterrupt: 

## Whitebox Adversarial Attack

In [ ]:
def evaluate_white_box(model, attack_fn, data_loader, epsilon, device):
    model.eval()
    correct = 0
    total = 0

    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        # 적대적 예제 생성
        adversarial_images = attack_fn(model, images, labels, epsilon)

        # 모델 예측
        outputs = model(adversarial_images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    robust_accuracy = 100 * correct / total
    print(f"Robust Accuracy: {robust_accuracy:.2f}%")
    return robust_accuracy

### Fast Gradient Sign Method (FGSM)

In [ ]:
def fgsm_attack(model, images, labels, epsilon):
    images.requires_grad = True
    outputs = model(images)
    loss = torch.nn.CrossEntropyLoss()(outputs, labels)
    model.zero_grad()
    loss.backward()
    grad = images.grad.data
    perturbed_images = images + epsilon * grad.sign()
    perturbed_images = torch.clamp(perturbed_images, 0, 1)
    return perturbed_images

### Momentum Iterative Method (MIM)

In [ ]:
def mim_attack(model, images, labels, epsilon, alpha, iters):
    perturbed_images = images.clone().detach()
    momentum = torch.zeros_like(images).to(images.device)

    for _ in range(iters):
        perturbed_images.requires_grad = True
        outputs = model(perturbed_images)
        loss = torch.nn.CrossEntropyLoss()(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad = perturbed_images.grad.data
        grad = grad / torch.mean(torch.abs(grad), dim=(1, 2, 3), keepdim=True)
        momentum = 0.9 * momentum + grad
        perturbed_images = perturbed_images + alpha * momentum.sign()
        perturbed_images = torch.clamp(perturbed_images, images - epsilon, images + epsilon)
        perturbed_images = torch.clamp(perturbed_images, 0, 1)
    return perturbed_images

## Test

In [ ]:
def evaluate_transferability(source_model, target_model, attack_fn, data_loader, epsilon, device):
    source_model.eval()
    target_model.eval()
    correct = 0
    total = 0

    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        # Source 모델에서 적대적 예제 생성
        adversarial_images = attack_fn(source_model, images, labels, epsilon)

        # Target 모델에서 평가
        outputs = target_model(adversarial_images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    transfer_accuracy = 100 * correct / total
    print(f"Transfer Accuracy: {transfer_accuracy:.2f}%")
    return transfer_accuracy

In [ ]:
# GPU 메모리 초기화
torch.cuda.empty_cache()

# PyTorch CUDA 메모리 사용량 확인
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1200 MiB |   1200 MiB |   1200 MiB |      0 B   |
|       from large pool |   1195 MiB |   1195 MiB |   1195 MiB |      0 B   |
|       from small pool |      5 MiB |      5 MiB |      5 MiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |   1200 MiB |   1200 MiB |   1200 MiB |      0 B   |
|       from large pool |   1195 MiB |   1195 MiB |   1195 MiB |

In [ ]:
# Transferability 테스트 실행
transferability_vit_to_cnn = evaluate_transferability(
    source_model=model_vit,
    target_model=model_cnn,
    attack_fn=fgsm_attack,
    data_loader=test_loader,
    epsilon=0.03,
    device=device
)

KeyboardInterrupt: 